# Assignment 3: Korean to English Translation

- Sequence to Sequence 모델의 대표적인 한국어-영어 번역을 Encoder-decoder, Attention, Convolution, 그리고 Transformers 기반으로 구현
- Pytorch Seq to Seq 모델 (https://github.com/bentrevett/pytorch-seq2seq) 참고로 하여 한국어와 영어의 형태소분석되고 의존관계로 되어 있는 파일을 프로세싱하여 두 언어의 parallel 데이터 쌍으로 만들고 이를 학습하여 모델별로 Perplexity가 어떻게 달라지는지 살펴 보고, 가장 성능이 좋은 모델을 근간으로 해서 Inference로 한국어 문장을 입력하면 대응되는 영어 번역이 출력될 수 있도록 구현
- 반드시 다음 세 모델에 대해서 PPL와 BLEU score가  다 체크되어야 함. **(Packed) Encoder-Decoder, Convolutional Seq to Seq, Transformers.
- **새로운 버전의 TorchText를 사용하여 코랩에서 실행가능하도록**

## Data
- 첨부된 ko-en-en.parse.syn은 330,974 한국어 문장에 대응되는 영어문장이 품사와 구문분석이 되어 있는 파일이고 ko-en-ko.parse.syn은 이에 대응되는 한국어 문장이 형태소와 구문분석이 되어 있는 파일이다.

(ROOT (S (NP (NNP Flight) (NNP 007)) (VP (MD will) (VP (VB stay) (PP (IN on) (NP (NP (DT the) (NN ground)) (PP (IN for) (NP (CD one) (NN hour))))))) (. .)))


<id 1>
<sent 1>
1       2       NP      777/SN
2       6       NP_SBJ  항공편/NNG|은/JX
3       4       NP      1/SN|시간/NNG
4       6       NP_AJT  동안/NNG
5       6       NP_AJT  지상/NNG|에/JKB
6       7       VP      머물/VV|게/EC
7       0       VP      되/VV|ㅂ니다/EF|./SF
</sent>
</id>

- 이 두 파일을 프로세싱하여 한-영 병행 데이터로 만들고 이를 학습 및 테스트 데이터로 사용한다.
- Hint: 구조화된 데이터를 프로세싱하기 위해서는 nltk의 모듈을 사용할 수 있다.

- 한국어 형태소 분석된 단위를 어절별로 결합할 수 있고, 분석된 채로 그대로 사용할 수도 있다.
- 두 언어의 어순을 비슷하게 데이터를 만들어 학습할 수도 있고, 번역의 성능을 높이기 위해 다양한 형태로 재구조화 할 수 있다.

# Packed Encoder-Decoder

In [ ]:
!pip install portalocker
import portalocker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import torch 

import numpy as np

import random
import math
import time

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torchtext.__version__

'0.15.2+cpu'

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

### 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 파일 불러오기

In [ ]:
PATH = "/content/drive/MyDrive/"

In [ ]:
ko_path = PATH+"ko-en.ko.parse"
en_path = PATH+"ko-en.en.parse.syn"

In [ ]:
import pandas as pd

In [ ]:
ko_lines = ""
with open(ko_path, "r", encoding='utf-8') as ko_file:
    for line in ko_file.readlines():
        ko_lines += line

In [ ]:
with open(en_path, "r", encoding='utf-8') as en_file:
    en_lines = en_file.readlines()

### 데이터 전처리

In [ ]:
from nltk import Tree

In [ ]:
# nltk의 Tree 모듈을 사용하여 필요한 정보 추출, <sos>, <eos> 토큰 각 문장에 넣어주기
full_en_text_list = []
for line in en_lines:
    sent = '<sos>' + ' '
    t = Tree.fromstring(line)
    for token in t.leaves():
      sent += token + ' '
    sent += '<eos>'
    full_en_text_list.append(sent)

In [ ]:
full_en_text_list[:100]

['<sos> Flight 007 will stay on the ground for one hour . <eos>',
 '<sos> Flight 017 will stay on the ground for three hours . <eos>',
 "<sos> I need 1,000 dollars in traveler 's checks . <eos>",
 '<sos> The official exchange rate is around 1,250 Won . <eos>',
 '<sos> Please give me three hundred dollar bills and twenty dollar bills for the rest . <eos>',
 '<sos> Can I have one hundred dollar bill and four fifty dollar bills ? <eos>',
 '<sos> Do you have change for $ 100 ? <eos>',
 "<sos> I 'd like to change 100 dollars . <eos>",
 "<sos> I 'd like to change $ 100 . <eos>",
 '<sos> Change 100 dollars . <eos>',
 "<sos> I 'd like to change 100 dollars . <eos>",
 '<sos> One hundred dollars . <eos>',
 "<sos> I want four 100 's , two 20 's , five 10 's and ten 1 's . <eos>",
 '<sos> 6 ten dollar bills and 8 five dollar bills , please . <eos>',
 '<sos> Could I have change for a one-hundred dollar bill ? <eos>',
 "<sos> I 'd like to change one hundred . <eos>",
 '<sos> I want four hundreds , t

In [ ]:
# 한 문장씩 나눈 리스트 만들기
ko_list = ko_lines.split("</id>")
for i in range(len(ko_list)):
    ko_list[i] = ko_list[i].split("\n")

ko_list[0]

['<id 1>',
 '<sent 1>',
 '1\t2\tNP\t777/SN',
 '2\t6\tNP_SBJ\t항공편/NNG|은/JX',
 '3\t4\tNP\t1/SN|시간/NNG',
 '4\t6\tNP_AJT\t동안/NNG',
 '5\t6\tNP_AJT\t지상/NNG|에/JKB',
 '6\t7\tVP\t머물/VV|게/EC',
 '7\t0\tVP\t되/VV|ㅂ니다/EF|./SF',
 '</sent>',
 '']

In [ ]:
# 필요한 정보만 추출
import re

pattern = r"[가-힣ㄱ-ㅎ]+|[0-9]+(?=\/SN)"

for i in range(len(ko_list)):
    for j in range(len(ko_list[i])):
        ko_list[i][j] = re.findall(pattern, ko_list[i][j])

In [ ]:
ko_list[:50]

[[[],
  [],
  ['777'],
  ['항공편', '은'],
  ['1', '시간'],
  ['동안'],
  ['지상', '에'],
  ['머물', '게'],
  ['되', 'ㅂ니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['777'],
  ['항공편', '은'],
  ['3', '시간'],
  ['동안'],
  ['지상', '에'],
  ['있', '겠', '습니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['1', '000', '달러'],
  ['여행자', '수표', '가'],
  ['필요', '하', 'ㅂ니다'],
  [],
  []],
 [[], [], [], [], ['1', '250', '원', '이'], ['공식'], ['환율', '이', 'ㅂ니다'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['3', '장', '과'],
  ['나머지', '는'],
  ['20', '달러', '권', '으로'],
  ['주', '시', 'ㅂ시오'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['한'],
  ['장', '과'],
  ['50', '달러'],
  ['4', '장', '으로'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '를'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[], [], [], [], ['100', '달러', '만'], ['바꾸', '어'], ['주', '시', '어요'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '만'],
  ['환전'],
  ['좀'],
  ['하', '아'],
  ['주', '시', '어요'],
  []

In [ ]:
# 문장으로 만들어서 리스트에 넣어주기, <sos>, <eos> 토큰 각 문장에 넣어주기
ko_text_list = []

for i in range(len(ko_list)):
    full_sent = '<sos>' + ' '
    for j in range(len(ko_list[i])):
        if ko_list[i][j]:
            for token in ko_list[i][j]:
                full_sent += token + ' '
    full_sent += '<eos>'

    ko_text_list.append(full_sent)

In [ ]:
ko_text_list[:100]

['<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>',
 '<sos> 1 000 달러 여행자 수표 가 필요 하 ㅂ니다 <eos>',
 '<sos> 1 250 원 이 공식 환율 이 ㅂ니다 <eos>',
 '<sos> 100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 <eos>',
 '<sos> 100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 <eos>',
 '<sos> 100 달러 를 바꾸 어 주 시 겠 어요 <eos>',
 '<sos> 100 달러 만 바꾸 어 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 좀 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 하 아 어 주 어 요 <eos>',
 '<sos> 100 달러 이 ㅂ니다 <eos>',
 '<sos> 100 달러 짜리 4 장 20 달러 짜리 2 장 10 달러 짜리 5 장 1 달러 짜리 10 장 으로 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 짜리 지폐 6 개 하 고 5 달러 짜리 지폐 8 개 로 바꾸 어 주 시 어요 <eos>',
 '<sos> 100 달러 짜리 지폐 를 잔돈 으로 바꾸 ㄹ 수 있 겠 습니까 <eos>',
 '<sos> 100 불 을 바꾸 겠 습니다 <eos>',
 '<sos> 100 불 짜리 4 매 20 불 짜리 3 매 10 불 짜리 3 매 5 불 짜리 1 매 그리고 1 불 짜리 5 매 원하 ㅂ니다 <eos>',
 '<sos> 100 마일리지 가 적립 되 었 습니다 <eos>',
 '<sos> 10 달러 5 장 과 20 달러 10 장 으로 부탁 하 ㅂ니다 <eos>',
 '<sos> 10 달러 정도 이 에요 <eos>',
 '<sos> 10 달러 지폐 7 장 1 달러 지폐 30 장 주 시 어요 <eos>',

### 한-영 병행 데이터 만들기

In [ ]:
print(full_en_text_list[:10])
print(ko_text_list[:10])

['<sos> Flight 007 will stay on the ground for one hour . <eos>', '<sos> Flight 017 will stay on the ground for three hours . <eos>', "<sos> I need 1,000 dollars in traveler 's checks . <eos>", '<sos> The official exchange rate is around 1,250 Won . <eos>', '<sos> Please give me three hundred dollar bills and twenty dollar bills for the rest . <eos>', '<sos> Can I have one hundred dollar bill and four fifty dollar bills ? <eos>', '<sos> Do you have change for $ 100 ? <eos>', "<sos> I 'd like to change 100 dollars . <eos>", "<sos> I 'd like to change $ 100 . <eos>", '<sos> Change 100 dollars . <eos>']
['<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>', '<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>', '<sos> 1 000 달러 여행자 수표 가 필요 하 ㅂ니다 <eos>', '<sos> 1 250 원 이 공식 환율 이 ㅂ니다 <eos>', '<sos> 100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 <eos>', '<sos> 100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 <eos>', '<sos> 100 달러 를 바꾸 어 주 시 겠 어요 <eos>', '<sos> 100 달러 만 바꾸 어 주 시 어요 <eos>', '<sos> 100 달러 만 환전 좀 하 아 주 시 어요 <eo

In [ ]:
parallel_data = list(zip(ko_text_list, full_en_text_list)) # src:한국어, trg:영어 순으로 넣어줌

In [ ]:
parallel_data[0]

('<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> Flight 007 will stay on the ground for one hour . <eos>')

In [ ]:
parallel_data[1]

('<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>',
 '<sos> Flight 017 will stay on the ground for three hours . <eos>')

In [ ]:
len(parallel_data)

330974

## 데이터 준비

In [ ]:
# 전체 데이터 사용 시 런타임 에러가 발생하여 100000개의 데이터만 사용
train_iter = parallel_data[0:70000]
valid_iter = parallel_data[70000:90000]
test_iter = parallel_data[90000:100000]

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_dataset = to_map_style_dataset(train_iter)
valid_dataset = to_map_style_dataset(valid_iter)
test_dataset = to_map_style_dataset(test_iter)

In [ ]:
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(file_iter, type = 0):
     
     if type == 0:
        for line in file_iter:
            yield line[0].split()
     else:
        for line in file_iter:
            yield line[1].split()

src_voc = build_vocab_from_iterator(yield_tokens(train_iter, 0), specials=["<unk>", "<pad>"])
trg_voc = build_vocab_from_iterator(yield_tokens(train_iter, 1), specials=["<unk>", "<pad>"])



In [ ]:
src_voc.set_default_index(src_voc["<unk>"])
trg_voc.set_default_index(trg_voc["<unk>"])

In [ ]:
print(len(src_voc))
print(len(trg_voc))

11379
12103


In [ ]:
src_pipeline = lambda x: src_voc(x.split())
trg_pipeline = lambda x: trg_voc(x.split())

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#collate_function: process the list of samples to form a batch. 
# https://androidkt.com/create-dataloader-with-collate_fn-for-variable-length-input-in-pytorch/


def custom_collate_fn(batch):
    src_list, trg_list, src_len = [], [], []
    for (_src, _trg) in batch:
         processed_src = torch.tensor(src_pipeline(_src), dtype=torch.int64)
         src_list.append(processed_src)
         src_len.append(len(processed_src))
         processed_trg = torch.tensor(trg_pipeline(_trg), dtype=torch.int64)
         trg_list.append(processed_trg)
    src_list = pad_sequence(src_list, padding_value = 1)
    trg_list = pad_sequence(trg_list, padding_value = 1)
    return src_list.to(device), trg_list.to(device), src_len

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=custom_collate_fn)

valid_dataloader = DataLoader(valid_dataset, batch_size=128,
                              shuffle=True, collate_fn=custom_collate_fn)

test_dataloader = DataLoader(test_dataset, batch_size=128,
                              shuffle=True, collate_fn=custom_collate_fn)

## Build the Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True) #GRU를 사용해서 cell state 받지 않음
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        #need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths = src_len, enforce_sorted = False)
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim] #nn.Linear(ddec_hid_dim, 1)

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask) #encoder의 output과 hidden state
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        ## 차원을 맞춰주기 위해 하는 과정 ##
        weighted = torch.bmm(a, encoder_outputs) #batch matrix multiplication
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
        ##

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)
        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [ ]:
INPUT_DIM = len(src_voc)
OUTPUT_DIM = len(trg_voc)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX =src_voc(['<pad>'])[0]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11379, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(12103, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=12103, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 34,145,351 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX =  trg_voc(['<pad>'])[0]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch[0], batch[2]
        trg = batch[1]
        
        optimizer.zero_grad()
        
        output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch[0], batch[2]
            trg = batch[1]

            output = model(src, src_len, trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Train the Model

In [ ]:
import torch.nn.functional as F
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 15s
	Train Loss: 3.826 | Train PPL:  45.880
	 Val. Loss: 4.638 |  Val. PPL: 103.313
Epoch: 02 | Time: 4m 19s
	Train Loss: 2.215 | Train PPL:   9.160
	 Val. Loss: 4.478 |  Val. PPL:  88.080
Epoch: 03 | Time: 4m 18s
	Train Loss: 1.699 | Train PPL:   5.466
	 Val. Loss: 4.462 |  Val. PPL:  86.642
Epoch: 04 | Time: 4m 16s
	Train Loss: 1.418 | Train PPL:   4.128
	 Val. Loss: 4.509 |  Val. PPL:  90.836
Epoch: 05 | Time: 4m 18s
	Train Loss: 1.225 | Train PPL:   3.405
	 Val. Loss: 4.595 |  Val. PPL:  99.006
Epoch: 06 | Time: 4m 17s
	Train Loss: 1.089 | Train PPL:   2.970
	 Val. Loss: 4.713 |  Val. PPL: 111.343
Epoch: 07 | Time: 4m 17s
	Train Loss: 0.998 | Train PPL:   2.713
	 Val. Loss: 4.836 |  Val. PPL: 125.933
Epoch: 08 | Time: 4m 19s
	Train Loss: 0.912 | Train PPL:   2.488
	 Val. Loss: 4.901 |  Val. PPL: 134.439
Epoch: 09 | Time: 4m 19s
	Train Loss: 0.842 | Train PPL:   2.320
	 Val. Loss: 4.946 |  Val. PPL: 140.550
Epoch: 10 | Time: 4m 18s
	Train Loss: 0.775 | Train PPL

## Test

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss = evaluate(model, test_dataloader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.530 | Test PPL:  92.745 |


## Inference

In [ ]:
trg_itos = trg_voc.get_itos()

In [ ]:
def translate_sentence(sentence, src_voc, trg_voc, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        tokens = [token for token in sentence.split()]
    else:
        tokens = [token for token in sentence]

    tokens = ['<sos>'] + tokens + ['<eos>']
        
    src_indexes = [src_voc[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_voc['<sos>']]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                #.unsqueeze(0)
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_voc['<eos>']:
            break
        
        #trg_indexes.append(pred_token)
    
    trg_tokens = [trg_itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']


In [ ]:
for sent in sen_list:
  translation, attention = translate_sentence(sent, src_voc, trg_voc, model, device)
  print(sent, translation)

모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 . ['You', 'are', 'placed', 'placed', 'placed', 'placed', 'placed', 'placed', 'placed', ',', 'and', 'placed', ',', 'and', ',', 'and', ',', 'and', ',', 'and', ',', 'and', ',', 'and', ',', 'and', ',', 'and', ',', '.', '<eos>']
미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ? ['Excuse', 'me', '.', 'I', "'d", 'like', 'to', 'check', 'with', 'a', '.', '.', 'I', "'d", 'like', 'to', 'change', 'my', 'bill', '.', '<eos>']
은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요 ['The', 'price', 'is', 'the', '.', '.', 'It', 'will', 'be', 'good', '.', '.', '<eos>']
아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ? ['I', "'d", 'like', 'to', 'fill', 'out', '.', 'I', "'d", 'like', 'to', '.', '.', 'Would', 'you', 'like', 'to', 'the', '?', '<eos>']
부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 . ['I', "'ve", 'a', 'a', 'the', 'previous', 'previous', 'previous', 'due', 'to', 'the', '.', '.', '<eos>']
변기 가 

## Bleu Score

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(data, src_voc, trg_voc, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = datum[0]
        trg = datum[1].split()

        trg.pop(0)
        trg.pop()
        
        pred_trg = translate_sentence(src, src_voc, trg_voc, model, device, max_len)

        pred_trgs.append(pred_trg)
        trgs.append(trg)
        
    return corpus_bleu(pred_trgs, trgs, weights=(1, 0, 0, 0))

In [ ]:
bleu_score = calculate_bleu(test_dataset, src_voc, trg_voc, model, device, max_len=50)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 37.47


# Convolutional Seq to Seq

## 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 파일 불러오기

In [ ]:
PATH = "/content/drive/MyDrive/"

In [ ]:
ko_path = PATH+"ko-en.ko.parse"
en_path = PATH+"ko-en.en.parse.syn"

In [ ]:
import pandas as pd

In [ ]:
ko_lines = ""
with open(ko_path, "r") as ko_file:
    for line in ko_file.readlines():
        ko_lines += line

In [ ]:
with open(en_path, "r") as en_file:
    en_lines = en_file.readlines()

## 데이터 전처리

In [ ]:
from nltk import Tree

In [ ]:
# nltk의 Tree 모듈을 사용하여 필요한 정보 추출, <sos>, <eos> 토큰 각 문장에 넣어주기
full_en_text_list = []
for line in en_lines:
    sent = '<sos>' + ' '
    t = Tree.fromstring(line)
    for token in t.leaves():
      sent += token + ' '
    sent += '<eos>'
    full_en_text_list.append(sent)

In [ ]:
full_en_text_list[:100]

['<sos> Flight 007 will stay on the ground for one hour . <eos>',
 '<sos> Flight 017 will stay on the ground for three hours . <eos>',
 "<sos> I need 1,000 dollars in traveler 's checks . <eos>",
 '<sos> The official exchange rate is around 1,250 Won . <eos>',
 '<sos> Please give me three hundred dollar bills and twenty dollar bills for the rest . <eos>',
 '<sos> Can I have one hundred dollar bill and four fifty dollar bills ? <eos>',
 '<sos> Do you have change for $ 100 ? <eos>',
 "<sos> I 'd like to change 100 dollars . <eos>",
 "<sos> I 'd like to change $ 100 . <eos>",
 '<sos> Change 100 dollars . <eos>',
 "<sos> I 'd like to change 100 dollars . <eos>",
 '<sos> One hundred dollars . <eos>',
 "<sos> I want four 100 's , two 20 's , five 10 's and ten 1 's . <eos>",
 '<sos> 6 ten dollar bills and 8 five dollar bills , please . <eos>',
 '<sos> Could I have change for a one-hundred dollar bill ? <eos>',
 "<sos> I 'd like to change one hundred . <eos>",
 '<sos> I want four hundreds , t

In [ ]:
# 한 문장씩 나눈 리스트 만들기
ko_list = ko_lines.split("</id>")
for i in range(len(ko_list)):
    ko_list[i] = ko_list[i].split("\n")

ko_list[0]

['<id 1>',
 '<sent 1>',
 '1\t2\tNP\t777/SN',
 '2\t6\tNP_SBJ\t항공편/NNG|은/JX',
 '3\t4\tNP\t1/SN|시간/NNG',
 '4\t6\tNP_AJT\t동안/NNG',
 '5\t6\tNP_AJT\t지상/NNG|에/JKB',
 '6\t7\tVP\t머물/VV|게/EC',
 '7\t0\tVP\t되/VV|ㅂ니다/EF|./SF',
 '</sent>',
 '']

In [ ]:
# 필요한 정보만 추출
import re

pattern = r"[가-힣ㄱ-ㅎ]+|[0-9]+(?=\/SN)"

for i in range(len(ko_list)):
    for j in range(len(ko_list[i])):
        ko_list[i][j] = re.findall(pattern, ko_list[i][j])

In [ ]:
ko_list[:50]

[[[],
  [],
  ['777'],
  ['항공편', '은'],
  ['1', '시간'],
  ['동안'],
  ['지상', '에'],
  ['머물', '게'],
  ['되', 'ㅂ니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['777'],
  ['항공편', '은'],
  ['3', '시간'],
  ['동안'],
  ['지상', '에'],
  ['있', '겠', '습니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['1', '000', '달러'],
  ['여행자', '수표', '가'],
  ['필요', '하', 'ㅂ니다'],
  [],
  []],
 [[], [], [], [], ['1', '250', '원', '이'], ['공식'], ['환율', '이', 'ㅂ니다'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['3', '장', '과'],
  ['나머지', '는'],
  ['20', '달러', '권', '으로'],
  ['주', '시', 'ㅂ시오'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['한'],
  ['장', '과'],
  ['50', '달러'],
  ['4', '장', '으로'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '를'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[], [], [], [], ['100', '달러', '만'], ['바꾸', '어'], ['주', '시', '어요'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '만'],
  ['환전'],
  ['좀'],
  ['하', '아'],
  ['주', '시', '어요'],
  []

In [ ]:
# 문장으로 만들어서 리스트에 넣어주기, <sos>, <eos> 토큰 각 문장에 넣어주기
ko_text_list = []

for i in range(len(ko_list)):
    full_sent = '<sos>' + ' '
    for j in range(len(ko_list[i])):
        if ko_list[i][j]:
            for token in ko_list[i][j]:
                full_sent += token + ' '
    full_sent += '<eos>'

    ko_text_list.append(full_sent)

In [ ]:
ko_text_list[:100]

['<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>',
 '<sos> 1 000 달러 여행자 수표 가 필요 하 ㅂ니다 <eos>',
 '<sos> 1 250 원 이 공식 환율 이 ㅂ니다 <eos>',
 '<sos> 100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 <eos>',
 '<sos> 100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 <eos>',
 '<sos> 100 달러 를 바꾸 어 주 시 겠 어요 <eos>',
 '<sos> 100 달러 만 바꾸 어 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 좀 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 하 아 어 주 어 요 <eos>',
 '<sos> 100 달러 이 ㅂ니다 <eos>',
 '<sos> 100 달러 짜리 4 장 20 달러 짜리 2 장 10 달러 짜리 5 장 1 달러 짜리 10 장 으로 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 짜리 지폐 6 개 하 고 5 달러 짜리 지폐 8 개 로 바꾸 어 주 시 어요 <eos>',
 '<sos> 100 달러 짜리 지폐 를 잔돈 으로 바꾸 ㄹ 수 있 겠 습니까 <eos>',
 '<sos> 100 불 을 바꾸 겠 습니다 <eos>',
 '<sos> 100 불 짜리 4 매 20 불 짜리 3 매 10 불 짜리 3 매 5 불 짜리 1 매 그리고 1 불 짜리 5 매 원하 ㅂ니다 <eos>',
 '<sos> 100 마일리지 가 적립 되 었 습니다 <eos>',
 '<sos> 10 달러 5 장 과 20 달러 10 장 으로 부탁 하 ㅂ니다 <eos>',
 '<sos> 10 달러 정도 이 에요 <eos>',
 '<sos> 10 달러 지폐 7 장 1 달러 지폐 30 장 주 시 어요 <eos>',

## 한-영 병행 데이터 만들기

In [ ]:
print(full_en_text_list[:10])
print(ko_text_list[:10])

['<sos> Flight 007 will stay on the ground for one hour . <eos>', '<sos> Flight 017 will stay on the ground for three hours . <eos>', "<sos> I need 1,000 dollars in traveler 's checks . <eos>", '<sos> The official exchange rate is around 1,250 Won . <eos>', '<sos> Please give me three hundred dollar bills and twenty dollar bills for the rest . <eos>', '<sos> Can I have one hundred dollar bill and four fifty dollar bills ? <eos>', '<sos> Do you have change for $ 100 ? <eos>', "<sos> I 'd like to change 100 dollars . <eos>", "<sos> I 'd like to change $ 100 . <eos>", '<sos> Change 100 dollars . <eos>']
['<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>', '<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>', '<sos> 1 000 달러 여행자 수표 가 필요 하 ㅂ니다 <eos>', '<sos> 1 250 원 이 공식 환율 이 ㅂ니다 <eos>', '<sos> 100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 <eos>', '<sos> 100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 <eos>', '<sos> 100 달러 를 바꾸 어 주 시 겠 어요 <eos>', '<sos> 100 달러 만 바꾸 어 주 시 어요 <eos>', '<sos> 100 달러 만 환전 좀 하 아 주 시 어요 <eo

In [ ]:
parallel_data = list(zip(ko_text_list, full_en_text_list)) # src:한국어, trg:영어 순으로 넣어줌

In [ ]:
parallel_data[0]

('<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> Flight 007 will stay on the ground for one hour . <eos>')

In [ ]:
parallel_data[1]

('<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>',
 '<sos> Flight 017 will stay on the ground for three hours . <eos>')

In [ ]:
len(parallel_data)

330974

## 데이터 준비

In [ ]:
!pip install portalocker
import portalocker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import random
import math
import time

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torchtext.__version__

'0.15.2+cpu'

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# 8:1:1로 train, valid, test 나눔
TRAIN_RATE = 0.8
VALID_RATE = 0.9

train_iter, valid_iter, test_iter = parallel_data[:int(len(parallel_data)*TRAIN_RATE)], parallel_data[int(len(parallel_data)*TRAIN_RATE):int(len(parallel_data)*VALID_RATE)], parallel_data[int(len(parallel_data)*VALID_RATE):]
train_dataset = to_map_style_dataset(train_iter)
valid_dataset = to_map_style_dataset(valid_iter)
test_dataset = to_map_style_dataset(test_iter)

In [ ]:
train_dataset[0]

('<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> Flight 007 will stay on the ground for one hour . <eos>')

In [ ]:
valid_dataset[0]

('<sos> 한식 을 좋아하 ㅂ니까 <eos>', '<sos> Do you like Korean food ? <eos>')

In [ ]:
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(file_iter, type = 0):
     if type == 0:
        for line in file_iter:
            yield line[0].split()
     else:
        for line in file_iter:
            yield line[1].split()

src_voc = build_vocab_from_iterator(yield_tokens(train_iter, 0), specials=["<unk>", "<pad>"])
trg_voc = build_vocab_from_iterator(yield_tokens(train_iter, 1), specials=["<unk>", "<pad>"])

In [ ]:
print(len(src_voc))
print(len(trg_voc))

29684
33549


In [ ]:
src_voc.set_default_index(src_voc["<unk>"])
trg_voc.set_default_index(trg_voc["<unk>"])

In [ ]:
src_pipeline = lambda x: src_voc(x.split())
trg_pipeline = lambda x: trg_voc(x.split())

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    src_list, trg_list, src_len = [], [], []
    for (_src, _trg) in batch:
        processed_src = torch.tensor(src_pipeline(_src), dtype=torch.int64)
        src_list.append(processed_src)
        src_len.append(len(processed_src))
        processed_trg = torch.tensor(trg_pipeline(_trg), dtype=torch.int64)
        trg_list.append(processed_trg)
    src_list = pad_sequence(src_list, batch_first=True, padding_value=1)
    trg_list = pad_sequence(trg_list, batch_first=True, padding_value=1)
    # batch_first=True로 설정해서 이후 forward에서 permute 안 해도 됨
    return src_list.to(device), trg_list.to(device), src_len

In [ ]:
BATCH_SIZE = 128

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=custom_collate_fn)

valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=custom_collate_fn)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=custom_collate_fn)

## Build the Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        assert kernel_size % 2 == 1, "Kernel size must be odd!"
        
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(input_dim, emb_dim) # token에 대한 정보
        self.pos_embedding = nn.Embedding(max_length, emb_dim) # positional 한 정보
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim) # emb_dim을 hid_dim으로 만들어줌
        self.hid2emb = nn.Linear(hid_dim, emb_dim) # hid_dim을 emb_dim으로 만들어줌
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size, 
                                              padding = (kernel_size - 1) // 2)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        
        #src = [batch size, src len]

        # src = src.permute(1,0) # 코드 삭제
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        #create position tensor
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [0, 1, 2, 3, ..., src len - 1]
        
        #pos = [batch size, src len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(src)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = pos_embedded = [batch size, src len, emb dim]

        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, src len, emb dim]
        
        #pass embedded through linear layer to convert from emb dim to hid dim

        conv_input = self.emb2hid(embedded)

        #conv_input = [batch size, src len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) # convolutional netword에서 요구하는 tensor size로 맞춰주기 위함
        
        #conv_input = [batch size, hid dim, src len]
        
        #begin convolutional blocks...
        
        for i, conv in enumerate(self.convs):
        
            #pass through convolutional layer
            conved = conv(self.dropout(conv_input))

            #conved = [batch size, 2 * hid dim, src len]

            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale

            #conved = [batch size, hid dim, src len]
            
            #set conv_input to conved for next loop iteration # 결과로 나온 것을 input으로 다시 들어갈 수 있도록 함
            conv_input = conved
        
        #...end convolutional blocks
        
        #permute and convert back to emb dim
        conved = self.hid2emb(conved.permute(0, 2, 1))
        
        #conved = [batch size, src len, emb dim]
        
        #elementwise sum output (conved) and input (embedded) to be used for attention
        combined = (conved + embedded) * self.scale
        
        #combined = [batch size, src len, emb dim]
        
        return conved, combined # conved: residual connection 적용X / combined: residual connection 적용O

In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 emb_dim, 
                 hid_dim, 
                 n_layers, 
                 kernel_size, 
                 dropout, 
                 trg_pad_idx, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.kernel_size = kernel_size
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
        self.scale = torch.sqrt(torch.FloatTensor([0.5])).to(device)
        
        self.tok_embedding = nn.Embedding(output_dim, emb_dim)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        
        self.emb2hid = nn.Linear(emb_dim, hid_dim)
        self.hid2emb = nn.Linear(hid_dim, emb_dim)
        
        self.attn_hid2emb = nn.Linear(hid_dim, emb_dim)
        self.attn_emb2hid = nn.Linear(emb_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim, output_dim)
        
        self.convs = nn.ModuleList([nn.Conv1d(in_channels = hid_dim, 
                                              out_channels = 2 * hid_dim, 
                                              kernel_size = kernel_size)
                                    for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
      
    def calculate_attention(self, embedded, conved, encoder_conved, encoder_combined):
        
        #embedded = [batch size, trg len, emb dim]
        #conved = [batch size, hid dim, trg len]
        #encoder_conved = encoder_combined = [batch size, src len, emb dim]
        
        #permute and convert back to emb dim

        conved_emb = self.attn_hid2emb(conved.permute(0, 2, 1))
        
        #conved_emb = [batch size, trg len, emb dim]
        
        combined = (conved_emb + embedded) * self.scale

        # 디버깅용 코드
        # print("0. conved_emb: ", end='')
        # print(conved_emb.size())

        # print("0. embedded: ", end='')
        # print(embedded.size())

        #combined = [batch size, trg len, emb dim]

        # print("1. combined: ", end='')
        # print(combined.size())

        # print("2. encoder_conved_permute: ", end='')
        # print(encoder_conved.permute(0,2,1).size())
                
        energy = torch.matmul(combined, encoder_conved.permute(0, 2, 1))

        # print("3. energy: ", end='')
        # print(energy.size())
        
        #energy = [batch size, trg len, src len]
        
        attention = F.softmax(energy, dim=2)
        
        #attention = [batch size, trg len, src len]
            
        attended_encoding = torch.matmul(attention, encoder_combined)
        
        #attended_encoding = [batch size, trg len, emd dim]
        
        #convert from emb dim -> hid dim
        attended_encoding = self.attn_emb2hid(attended_encoding)
        
        #attended_encoding = [batch size, trg len, hid dim]
        
        #apply residual connection
        attended_combined = (conved + attended_encoding.permute(0, 2, 1)) * self.scale
        
        #attended_combined = [batch size, hid dim, trg len]
        
        return attention, attended_combined
        
    def forward(self, trg, encoder_conved, encoder_combined):
        
        #trg = [batch size, trg len]
        #encoder_conved = encoder_combined = [batch size, src len, emb dim]

        # trg = trg.permute(1,0) # 코드 삭제
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
            
        #create position tensor
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, trg len]
        
        #embed tokens and positions
        tok_embedded = self.tok_embedding(trg)
        pos_embedded = self.pos_embedding(pos)
        
        #tok_embedded = [batch size, trg len, emb dim]
        #pos_embedded = [batch size, trg len, emb dim]
        
        #combine embeddings by elementwise summing
        embedded = self.dropout(tok_embedded + pos_embedded)
        
        #embedded = [batch size, trg len, emb dim]
        
        #pass embedded through linear layer to go through emb dim -> hid dim
        conv_input = self.emb2hid(embedded)
        
        #conv_input = [batch size, trg len, hid dim]
        
        #permute for convolutional layer
        conv_input = conv_input.permute(0, 2, 1) 
        
        #conv_input = [batch size, hid dim, trg len]
        
        batch_size = conv_input.shape[0]
        hid_dim = conv_input.shape[1]
        
        for i, conv in enumerate(self.convs):
        
            #apply dropout
            conv_input = self.dropout(conv_input)
        
            #need to pad so decoder can't "cheat"
            padding = torch.zeros(batch_size, 
                                  hid_dim, 
                                  self.kernel_size - 1).fill_(self.trg_pad_idx).to(self.device)
                
            padded_conv_input = torch.cat((padding, conv_input), dim = 2)
        
            #padded_conv_input = [batch size, hid dim, trg len + kernel size - 1]
        
            #pass through convolutional layer
            conved = conv(padded_conv_input)

            #conved = [batch size, 2 * hid dim, trg len]
            
            #pass through GLU activation function
            conved = F.glu(conved, dim = 1)

            #conved = [batch size, hid dim, trg len]
            
            #calculate attention
            attention, conved = self.calculate_attention(embedded, 
                                                         conved, 
                                                         encoder_conved, 
                                                         encoder_combined)
            
            #attention = [batch size, trg len, src len]
            
            #apply residual connection
            conved = (conved + conv_input) * self.scale
            
            #conved = [batch size, hid dim, trg len]
            
            #set conv_input to conved for next loop iteration
            conv_input = conved
            
        conved = self.hid2emb(conved.permute(0, 2, 1))
         
        #conved = [batch size, trg len, emb dim]
            
        output = self.fc_out(self.dropout(conved))
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len - 1] (<eos> token sliced off the end)
           
        #calculate z^u (encoder_conved) and (z^u + e) (encoder_combined)
        #encoder_conved is output from final encoder conv. block
        #encoder_combined is encoder_conved plus (elementwise) src embedding plus 
        #  positional embeddings 
        encoder_conved, encoder_combined = self.encoder(src)
            
        #encoder_conved = [batch size, src len, emb dim]
        #encoder_combined = [batch size, src len, emb dim]
        
        #calculate predictions of next words
        #output is a batch of predictions for each word in the trg sentence
        #attention a batch of attention scores across the src sentence for 
        #  each word in the trg sentence
        output, attention = self.decoder(trg, encoder_conved, encoder_combined)
        
        #output = [batch size, trg len - 1, output dim]
        #attention = [batch size, trg len - 1, src len]
        
        return output, attention

In [ ]:
INPUT_DIM = len(src_voc)
OUTPUT_DIM = len(trg_voc)
EMB_DIM = 256
HID_DIM = 512 # each conv. layer has 2 * hid_dim filters
ENC_LAYERS = 10 # number of conv. blocks in encoder
DEC_LAYERS = 10 # number of conv. blocks in decoder
ENC_KERNEL_SIZE = 3 # must be odd!
DEC_KERNEL_SIZE = 3 # can be even or odd
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25
TRG_PAD_IDX = trg_voc(['<pad>'])[0]

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, ENC_LAYERS, ENC_KERNEL_SIZE, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, DEC_LAYERS, DEC_KERNEL_SIZE, DEC_DROPOUT, TRG_PAD_IDX, device)

model = Seq2Seq(enc, dec).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 57,127,437 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0]
        trg = batch[1]
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
        
        output_dim = output.shape[-1]
        
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1]

            output, _ = model(src, trg[:,:-1])
        
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]

            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)

            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Train the Model

In [ ]:
N_EPOCHS = 5 
# 너무 오래 걸리고, 과적합이 발생하여 epoch를 10까지 돌리면 중간에 loss가 늘어나는 문제 발생
# 또한, epoch 5 즈음부터는 loss가 줄어드는 비율이 월등히 낮아지는 등의 학습에 큰 영향을 주는 일이 없는 것 같아 epoch 5까지만 돌림
CLIP = 0.1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 12m 17s
	Train Loss: 3.186 | Train PPL:  24.180
	 Val. Loss: 2.744 |  Val. PPL:  15.543
Epoch: 02 | Time: 12m 6s
	Train Loss: 2.266 | Train PPL:   9.646
	 Val. Loss: 2.353 |  Val. PPL:  10.521
Epoch: 03 | Time: 12m 5s
	Train Loss: 2.009 | Train PPL:   7.458
	 Val. Loss: 2.144 |  Val. PPL:   8.533
Epoch: 04 | Time: 12m 7s
	Train Loss: 1.851 | Train PPL:   6.365
	 Val. Loss: 1.987 |  Val. PPL:   7.297
Epoch: 05 | Time: 12m 8s
	Train Loss: 1.732 | Train PPL:   5.653
	 Val. Loss: 1.876 |  Val. PPL:   6.529


## Test

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss = evaluate(model, test_dataloader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 1.791 | Test PPL:   5.995 |


## Inference

In [ ]:
trg_itos = trg_voc.get_itos()

In [ ]:
def translate_sentence(sentence, src_voc, trg_voc, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        tokens = [token for token in sentence.split()]
    else:
        tokens = [token for token in sentence]

    tokens = ['<sos>'] + tokens + ['<eos>']
        
    src_indexes = [src_voc[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_conved, encoder_combined = model.encoder(src_tensor)

    trg_indexes = [trg_voc['<sos>']]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, encoder_conved, encoder_combined)
                
        pred_token = output.argmax(2)[:,-1].item()

        if pred_token == trg_voc['<eos>']:
            break
        
        trg_indexes.append(pred_token)
    
    trg_tokens = [trg_itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
for sent in sen_list:
    translation, attention = translate_sentence(sent, src_voc, trg_voc, model, device)
    print(f'predicted trg = {translation}')

predicted trg = ['I', 'have', 'to', 'put', 'them', 'of', 'liquids', ',', 'and', 'aerosols', ',', 'and', 'aerosols', ',', 'and', 'aerosols', ',', 'and', 'a', 'plastic', 'bag', '.']
predicted trg = ['I', "'m", 'sorry', ',', 'but', 'I', "'d", 'like', 'to', 'have', 'a', 'little', 'meeting', 'of', 'the', 'speed', '.', 'May', 'I', 'have', 'a', 'ticket', 'for', 'a', 'few', 'distance', '?']
predicted trg = ['The', 'bank', 'is', 'too', 'far', 'from', 'the', 'bank', '.', 'I', 'need', 'a', 'little', '.', 'I', 'need', 'to', 'be', 'a', 'little', '.']
predicted trg = ['I', 'think', 'I', 'have', 'lost', 'lost', 'the', 'lost', ',', 'but', 'I', 'have', 'to', 'have', 'a', 'lost', 'of', 'the', 'lost', '.', 'Do', 'you', 'want', 'to', 'have', 'a', 'lost', 'of', 'the', 'lost', '?', 'I', 'need', 'to', 'be', 'out', 'with', 'the', 'lost', '.', 'Do', 'you', 'have', 'to', 'put', 'the', 'lost', 'lost', 'or', 'I']
predicted trg = ['I', "'m", 'going', 'to', 'get', 'a', 'few', 'minute', 'call', 'from', 'Busan', ',',

## Bleu Score

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(data, src_voc, trg_voc, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = datum[0]
        trg = datum[1].split()

        # <sos>, <eos> 제거
        trg.pop(0)
        trg.pop()
        
        pred_trg = translate_sentence(src, src_voc, trg_voc, model, device)

        pred_trgs.append(pred_trg)
        trgs.append(trg)
        
    return corpus_bleu(pred_trgs, trgs, weights=(1, 0, 0, 0))

In [ ]:
bleu_score = calculate_bleu(test_dataset, src_voc, trg_voc, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 48.12


# Transformers

transformers 모델만 특이하게 bleu score가 낮은 문제를 겪고 있었는데, dropout, normalization 등을 실행해보고 해결되지 않아 조교님께 질문드렸더니 다음과 같은 대처법을 알려주셨습니다.

* 데이터셋이 배치 단위로 올바르게 훈련이 되는 지 뽑아보기

: dataloader를 생성하고 대표로 train_dataset에서 첫번째 batch를 뽑아 형태를 확인하였으며, 문제가 없었습니다.

* 훈련 과정에서 validation set에 대한 loss 수렴이 잘 이뤄지고 있는지 판단하기

: validation set에 대한 loss 수렴은 잘 이뤄졌습니다.

* 모든 훈련 스텝에 문제가 없다면 bleu score를 측정하는 내부 코드를 살펴보기

: bleu score를 측정하는 내부 코드를 확인하였으며, 이는 학우들과 상의하여 수정한 코드로, 똑같은 코드에 대해서 학우들은 정상적으로 bleu score가 산출되었습니다.



말씀해주신 대처법을 포함해 다양한 방안에서 문제를 해결해보고자 노력하였는데, 코드에서 문제를 발견하지 못했고, transformer의 경우에만 특이하게 bleu score가 낮은 원인을 파악하지 못하였습니다.

어떤 부분이 문제가 되어 bleu score가 낮게 나온 것인지 알려주신다면 학습에 큰 도움이 될 것이라 생각합니다. 감사합니다. kyky8392@snu.ac.kr




## 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 파일 불러오기

In [ ]:
PATH = "/content/drive/MyDrive/"

In [ ]:
ko_path = PATH+"ko-en.ko.parse"
en_path = PATH+"ko-en.en.parse.syn"

In [ ]:
import pandas as pd

In [ ]:
ko_lines = ""
with open(ko_path, "r") as ko_file:
    for line in ko_file.readlines():
        ko_lines += line

In [ ]:
with open(en_path, "r") as en_file:
    en_lines = en_file.readlines()

## 데이터 전처리

In [ ]:
from nltk import Tree

In [ ]:
# nltk의 Tree 모듈을 사용하여 필요한 정보 추출, <sos>, <eos> 토큰 각 문장에 넣어주기
full_en_text_list = []
for line in en_lines:
    sent = '<sos>' + ' '
    t = Tree.fromstring(line)
    for token in t.leaves():
      sent += token + ' '
    sent += '<eos>'
    full_en_text_list.append(sent)

In [ ]:
full_en_text_list[:100]

['<sos> Flight 007 will stay on the ground for one hour . <eos>',
 '<sos> Flight 017 will stay on the ground for three hours . <eos>',
 "<sos> I need 1,000 dollars in traveler 's checks . <eos>",
 '<sos> The official exchange rate is around 1,250 Won . <eos>',
 '<sos> Please give me three hundred dollar bills and twenty dollar bills for the rest . <eos>',
 '<sos> Can I have one hundred dollar bill and four fifty dollar bills ? <eos>',
 '<sos> Do you have change for $ 100 ? <eos>',
 "<sos> I 'd like to change 100 dollars . <eos>",
 "<sos> I 'd like to change $ 100 . <eos>",
 '<sos> Change 100 dollars . <eos>',
 "<sos> I 'd like to change 100 dollars . <eos>",
 '<sos> One hundred dollars . <eos>',
 "<sos> I want four 100 's , two 20 's , five 10 's and ten 1 's . <eos>",
 '<sos> 6 ten dollar bills and 8 five dollar bills , please . <eos>',
 '<sos> Could I have change for a one-hundred dollar bill ? <eos>',
 "<sos> I 'd like to change one hundred . <eos>",
 '<sos> I want four hundreds , t

In [ ]:
# 한 문장씩 나눈 리스트 만들기
ko_list = ko_lines.split("</id>")
for i in range(len(ko_list)):
    ko_list[i] = ko_list[i].split("\n")

ko_list[0]

['<id 1>',
 '<sent 1>',
 '1\t2\tNP\t777/SN',
 '2\t6\tNP_SBJ\t항공편/NNG|은/JX',
 '3\t4\tNP\t1/SN|시간/NNG',
 '4\t6\tNP_AJT\t동안/NNG',
 '5\t6\tNP_AJT\t지상/NNG|에/JKB',
 '6\t7\tVP\t머물/VV|게/EC',
 '7\t0\tVP\t되/VV|ㅂ니다/EF|./SF',
 '</sent>',
 '']

In [ ]:
# 필요한 정보만 추출
import re

pattern = r"[가-힣ㄱ-ㅎ]+|[0-9]+(?=\/SN)"

for i in range(len(ko_list)):
    for j in range(len(ko_list[i])):
        ko_list[i][j] = re.findall(pattern, ko_list[i][j])

In [ ]:
ko_list[:50]

[[[],
  [],
  ['777'],
  ['항공편', '은'],
  ['1', '시간'],
  ['동안'],
  ['지상', '에'],
  ['머물', '게'],
  ['되', 'ㅂ니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['777'],
  ['항공편', '은'],
  ['3', '시간'],
  ['동안'],
  ['지상', '에'],
  ['있', '겠', '습니다'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['1', '000', '달러'],
  ['여행자', '수표', '가'],
  ['필요', '하', 'ㅂ니다'],
  [],
  []],
 [[], [], [], [], ['1', '250', '원', '이'], ['공식'], ['환율', '이', 'ㅂ니다'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['3', '장', '과'],
  ['나머지', '는'],
  ['20', '달러', '권', '으로'],
  ['주', '시', 'ㅂ시오'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러'],
  ['한'],
  ['장', '과'],
  ['50', '달러'],
  ['4', '장', '으로'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '를'],
  ['바꾸', '어'],
  ['주', '시', '겠', '어요'],
  [],
  []],
 [[], [], [], [], ['100', '달러', '만'], ['바꾸', '어'], ['주', '시', '어요'], [], []],
 [[],
  [],
  [],
  [],
  ['100', '달러', '만'],
  ['환전'],
  ['좀'],
  ['하', '아'],
  ['주', '시', '어요'],
  []

In [ ]:
# 문장으로 만들어서 리스트에 넣어주기, <sos>, <eos> 토큰 각 문장에 넣어주기
ko_text_list = []

for i in range(len(ko_list)):
    full_sent = '<sos>' + ' '
    for j in range(len(ko_list[i])):
        if ko_list[i][j]:
            for token in ko_list[i][j]:
                full_sent += token + ' '
    full_sent += '<eos>'

    ko_text_list.append(full_sent)

In [ ]:
ko_text_list[:100]

['<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>',
 '<sos> 1 000 달러 여행자 수표 가 필요 하 ㅂ니다 <eos>',
 '<sos> 1 250 원 이 공식 환율 이 ㅂ니다 <eos>',
 '<sos> 100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 <eos>',
 '<sos> 100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 <eos>',
 '<sos> 100 달러 를 바꾸 어 주 시 겠 어요 <eos>',
 '<sos> 100 달러 만 바꾸 어 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 좀 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 만 환전 하 아 어 주 어 요 <eos>',
 '<sos> 100 달러 이 ㅂ니다 <eos>',
 '<sos> 100 달러 짜리 4 장 20 달러 짜리 2 장 10 달러 짜리 5 장 1 달러 짜리 10 장 으로 하 아 주 시 어요 <eos>',
 '<sos> 100 달러 짜리 지폐 6 개 하 고 5 달러 짜리 지폐 8 개 로 바꾸 어 주 시 어요 <eos>',
 '<sos> 100 달러 짜리 지폐 를 잔돈 으로 바꾸 ㄹ 수 있 겠 습니까 <eos>',
 '<sos> 100 불 을 바꾸 겠 습니다 <eos>',
 '<sos> 100 불 짜리 4 매 20 불 짜리 3 매 10 불 짜리 3 매 5 불 짜리 1 매 그리고 1 불 짜리 5 매 원하 ㅂ니다 <eos>',
 '<sos> 100 마일리지 가 적립 되 었 습니다 <eos>',
 '<sos> 10 달러 5 장 과 20 달러 10 장 으로 부탁 하 ㅂ니다 <eos>',
 '<sos> 10 달러 정도 이 에요 <eos>',
 '<sos> 10 달러 지폐 7 장 1 달러 지폐 30 장 주 시 어요 <eos>',

## 한-영 병행 데이터 만들기

In [ ]:
print(full_en_text_list[:10])
print(ko_text_list[:10])

['<sos> Flight 007 will stay on the ground for one hour . <eos>', '<sos> Flight 017 will stay on the ground for three hours . <eos>', "<sos> I need 1,000 dollars in traveler 's checks . <eos>", '<sos> The official exchange rate is around 1,250 Won . <eos>', '<sos> Please give me three hundred dollar bills and twenty dollar bills for the rest . <eos>', '<sos> Can I have one hundred dollar bill and four fifty dollar bills ? <eos>', '<sos> Do you have change for $ 100 ? <eos>', "<sos> I 'd like to change 100 dollars . <eos>", "<sos> I 'd like to change $ 100 . <eos>", '<sos> Change 100 dollars . <eos>']
['<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>', '<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>', '<sos> 1 000 달러 여행자 수표 가 필요 하 ㅂ니다 <eos>', '<sos> 1 250 원 이 공식 환율 이 ㅂ니다 <eos>', '<sos> 100 달러 3 장 과 나머지 는 20 달러 권 으로 주 시 ㅂ시오 <eos>', '<sos> 100 달러 한 장 과 50 달러 4 장 으로 바꾸 어 주 시 겠 어요 <eos>', '<sos> 100 달러 를 바꾸 어 주 시 겠 어요 <eos>', '<sos> 100 달러 만 바꾸 어 주 시 어요 <eos>', '<sos> 100 달러 만 환전 좀 하 아 주 시 어요 <eo

In [ ]:
parallel_data = list(zip(ko_text_list, full_en_text_list)) # src:한국어, trg:영어 순으로 넣어줌

In [ ]:
parallel_data[0]

('<sos> 777 항공편 은 1 시간 동안 지상 에 머물 게 되 ㅂ니다 <eos>',
 '<sos> Flight 007 will stay on the ground for one hour . <eos>')

In [ ]:
parallel_data[1]

('<sos> 777 항공편 은 3 시간 동안 지상 에 있 겠 습니다 <eos>',
 '<sos> Flight 017 will stay on the ground for three hours . <eos>')

## 데이터 준비

In [ ]:
!pip install portalocker
import portalocker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.datasets import Multi30k

import spacy
import numpy as np

import random
import math
import time

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torchtext.__version__

'0.15.2+cpu'

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
#8:1:1 로 train, valid, test를 나눔

train_iter = parallel_data[:int(len(parallel_data)*0.8)]
valid_iter = parallel_data[int(len(parallel_data)*0.8):int(len(parallel_data)*0.9)]
test_iter = parallel_data[int(len(parallel_data)*0.9):]

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_dataset = to_map_style_dataset(train_iter)
valid_dataset = to_map_style_dataset(valid_iter)
test_dataset = to_map_style_dataset(test_iter)

In [ ]:
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(file_iter, type = 0):
     
     if type == 0:
        for line in file_iter:
            yield line[0].split()
     else:
        for line in file_iter:
            yield line[1].split()

src_voc = build_vocab_from_iterator(yield_tokens(train_iter, 0), specials=["<unk>", "<pad>"])
trg_voc = build_vocab_from_iterator(yield_tokens(train_iter, 1), specials=["<unk>", "<pad>"])

In [ ]:
print(len(src_voc))
print(len(trg_voc))

29522
33544


In [ ]:
src_voc.set_default_index(src_voc["<unk>"])
trg_voc.set_default_index(trg_voc["<unk>"])

In [ ]:
src_pipeline = lambda x: src_voc(x.split())
trg_pipeline = lambda x: trg_voc(x.split())

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    src_list, trg_list, src_len = [], [], []
    for (_src, _trg) in batch:
        processed_src = torch.tensor(src_pipeline(_src), dtype=torch.int64)
        src_list.append(processed_src)
        src_len.append(len(processed_src))
        processed_trg = torch.tensor(trg_pipeline(_trg), dtype=torch.int64)
        trg_list.append(processed_trg)
    
    src_list = pad_sequence(src_list, padding_value=1, batch_first=True)
    trg_list = pad_sequence(trg_list, padding_value=1, batch_first=True)
    
    return src_list.to(device), trg_list.to(device), src_len

In [ ]:
BATCH_SIZE = 128

# DataLoader를 사용하여 데이터셋을 배치 단위로 로드

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=custom_collate_fn)

valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=custom_collate_fn)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=custom_collate_fn)

데이터셋이 배치 단위로 올바르게 훈련이 되는 지 뽑아보기

In [ ]:
# 대표로 train_dataloader 첫 번째 배치를 가져오기
batch = next(iter(train_dataloader))

print(batch[0]) #src
print(batch[1]) #trg
print(len(batch[0])) #src 개수, 128개여야 함
print(len(batch[1])) #trg 개수, 128개여야 함

tensor([[   3,   89,  941,  ...,    1,    1,    1],
        [   3,  470,  411,  ...,    1,    1,    1],
        [   3,   65,   72,  ...,    1,    1,    1],
        ...,
        [   3,  715,    4,  ...,    1,    1,    1],
        [   3, 5348,   56,  ...,    1,    1,    1],
        [   3,   39,    8,  ...,    1,    1,    1]], device='cuda:0')
tensor([[   3,    6,  265,  ...,    1,    1,    1],
        [   3,   60,    6,  ...,    1,    1,    1],
        [   3,  373,   17,  ...,    1,    1,    1],
        ...,
        [   3,   40,   16,  ...,    1,    1,    1],
        [   3,   42, 3282,  ...,    1,    1,    1],
        [   3,   81, 2481,  ...,    1,    1,    1]], device='cuda:0')
128
128


 batch 형태가 정상적이며, 데이터셋이 배치 단위로 올바르게 훈련이 되어야합니다.

## Build the Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [ ]:
import torch.nn.functional as F

#overfitting을 의심해보고 multi-head attention layer에서 배치 정규화를 시도

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
        self.batch_norm = nn.BatchNorm1d(hid_dim)  # 배치 정규화
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
        
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        # 배치 정규화를 적용
        x_norm = self.batch_norm(x.permute(0, 2, 1))
        x_norm = x_norm.permute(0, 2, 1)
        
        x = self.fc_o(x_norm)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [ ]:
INPUT_DIM = len(src_voc)
OUTPUT_DIM = len(trg_voc)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.25 #train/valid/test/inference는 성능이 좋은데 이상하게 bleu score만 성능이 낮아 overfitting을 의심해보고 해결 방안으로 dropout 비율을 0.1에서 0.25로 늘려봄
DEC_DROPOUT = 0.25

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [ ]:
SRC_PAD_IDX = 1
TRG_PAD_IDX = 1

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 28,819,213 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
model.apply(initialize_weights);

In [ ]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch[0]
        trg = batch[1]
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch[0]
            trg = batch[1]

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

훈련 과정에서 validation set에 대한 loss 수렴이 잘 이뤄지고 있는지 판단하기

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 21s
	Train Loss: 3.030 | Train PPL:  20.695
	 Val. Loss: 2.555 |  Val. PPL:  12.867
Epoch: 02 | Time: 4m 16s
	Train Loss: 1.899 | Train PPL:   6.681
	 Val. Loss: 2.000 |  Val. PPL:   7.391
Epoch: 03 | Time: 4m 17s
	Train Loss: 1.546 | Train PPL:   4.695
	 Val. Loss: 1.701 |  Val. PPL:   5.479
Epoch: 04 | Time: 4m 17s
	Train Loss: 1.351 | Train PPL:   3.862
	 Val. Loss: 1.498 |  Val. PPL:   4.471
Epoch: 05 | Time: 4m 16s
	Train Loss: 1.216 | Train PPL:   3.373
	 Val. Loss: 1.353 |  Val. PPL:   3.869
Epoch: 06 | Time: 4m 16s
	Train Loss: 1.115 | Train PPL:   3.050
	 Val. Loss: 1.238 |  Val. PPL:   3.449
Epoch: 07 | Time: 4m 16s
	Train Loss: 1.035 | Train PPL:   2.814
	 Val. Loss: 1.158 |  Val. PPL:   3.184
Epoch: 08 | Time: 4m 17s
	Train Loss: 0.971 | Train PPL:   2.642
	 Val. Loss: 1.088 |  Val. PPL:   2.968
Epoch: 09 | Time: 4m 16s
	Train Loss: 0.919 | Train PPL:   2.506
	 Val. Loss: 1.033 |  Val. PPL:   2.808
Epoch: 10 | Time: 4m 16s
	Train Loss: 0.876 | Train PPL

훈련 과정에서 validation set에 대한 loss 수렴이 잘 이뤄지고 있습니다.

## Test

In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss = evaluate(model, test_dataloader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 0.907 | Test PPL:   2.476 |


## Inference

In [ ]:
trg_itos = trg_voc.get_itos()

In [ ]:
def translate_sentence(sentence, src_voc, trg_voc, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        tokens = [token for token in sentence.split()]
    else:
        tokens = [token for token in sentence]

    tokens = ['<sos>'] + tokens + ['<eos>']
        
    src_indexes = [src_voc[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    src_mask = model.make_src_mask(src_tensor) #transformer 모델에서는 src에 mask 적용

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_voc['<sos>']]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor) #trg에도 mask 적용

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
                
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_voc['<eos>']:
            break
    
    trg_tokens = [trg_itos[i] for i in trg_indexes]
    
    return trg_tokens[1:-1]

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
for sent in sen_list:
  print(sent)
  print(translate_sentence(sent, src_voc, trg_voc, model, device, max_len = 50))

모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .
['All', 'liquids', ',', 'gels', 'and', 'aerosols', 'must', 'be', 'placed', 'in', 'a', 'single', 'and', 'single', 'and', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'single', 'and', 'a', 'single', 'and']
미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?
['I', "'m", 'sorry', ',', 'but', 'I', "'d", 'like', 'to', 'change', 'my', 'ticket', 'to', 'the', 'child', '.', 'Could', 'you', 'give', 'me', 'a', 'ticket', 'to', 'the', 'double', 'decker', 'bus', 'number', 'is', 'a', 'ticket', 'to', 'the', 'movie', 'ticket', ',', 'please', 'change', 'for', 'me', 'a', 'ticket', 'to', 'the', 'movie', 'beer', ',', 'and', 'reserve']
은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요
['The', 'bank', 'is', 'too', 'far', '.', 'Do', 'you', 'want', 'to', 'pay', 'too', '

## BLEU Score

모든 훈련 스텝에 문제가 없다면 bleu score를 측정하는 내부 코드를 살펴보기

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(data, src_voc, trg_voc, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data: #test_dataset의 원소 하나하나에 대해
        
        src = datum[0]
        trg = datum[1].split() #trg는 문장 형태로 되어있으므로 단어별로 토큰화하여 하나의 리스트로 만들기, split()을 이용

        trg.pop(0) #<sos> 제거
        trg.pop() #<eos> 제거
        
        pred_trg = translate_sentence(src, src_voc, trg_voc, model, device, max_len)
        #위의 inference 형태처럼 pred_trg는 단어별로 토큰화되어 하나의 리스트 안에 들어가 있음

        #단어별로 토큰화된 리스트를 pred_trgs, trgs에 넣어줌
        pred_trgs.append(pred_trg)
        trgs.append(trg)
        
    #단어별로 토큰화된 리스트'들' 전체(pred_trgs, trgs)를 대상으로 corpus_bleu 메서드를 이용해 bleu score를 구함.
    return corpus_bleu(pred_trgs, trgs, weights=(1, 0, 0, 0))

calculate_bleu 함수 내부 코드를 확인하였으며, 이는 학우들과 상의하며 수정한 코드로, 똑같은 코드에 대해서 학우들은 정상적으로 bleu score가 산출되었기에 bleu 함수에는 문제가 없다고 판단했습니다.

In [ ]:
bleu_score = calculate_bleu(test_dataset, src_voc, trg_voc, model, device, max_len=50)

print(f'BLEU score = {bleu_score*100:.2f}') #train/valid/test/inference는 성능이 좋은데 이상하게 bleu score만 성능이 낮음. overfitting을 의심해보고 배치 정규화/dropout 비율 조정 등 다양한 시도를 해보았으나 안타깝게 해결되지 않음.

BLEU score = 18.69


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

훈련 스텝과 bleu score 내부 코드를 확인해보았으나 특이하게 transformer에 대해서만 bleu score가 낮게 나옵니다.

# 결론

## Packed Encoder-Decoder 모델
- 가장 느렸다.
- Bleu score 37.47이다.
- Inference는 그렇게 잘 반영되는 것 같지는 않다.

## Convolutional Seq to Seq 모델
- Packed Encoder-Decoder 모델보다는 빨랐지만, Transformers 모델보다는 느렸다.
- Bleu score 48.12로 가장 높았다.
- Inference는 어느 정도 괜찮게 나오는 것으로 보인다.

## Transformers 모델
- 가장 빨랐다. (Convolutional Seq to Seq 모델보다 약 4배 정도 빨랐다.)
- Bleu score 18.69로 예상보다 낮게 나왔다. (훈련 스텝과 bleu score 내부 코드를 확인해보았으나 특이하게 transformer에 대해서만 bleu score가 낮게 나온다.)
- 하지만 Inference는 잘 나오는 것으로 보이고, loss, perplexity도 전체적으로 가장 낮게 나온 것으로 보아 학습은 제대로 이루어진 것 같다.

## 요약

Bleu score가 가장 높게 나온 모델은 CNN seq to seq 모델이다. 이 모델의 inference를 다시 한 번 보여주자면 아래와 같다.


In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
for sent in sen_list:
    translation, attention = translate_sentence(sent, src_voc, trg_voc, model, device)
    print(f'predicted trg = {translation}')

predicted trg = ['I', 'have', 'to', 'put', 'them', 'of', 'liquids', ',', 'and', 'aerosols', ',', 'and', 'aerosols', ',', 'and', 'aerosols', ',', 'and', 'a', 'plastic', 'bag', '.']
predicted trg = ['I', "'m", 'sorry', ',', 'but', 'I', "'d", 'like', 'to', 'have', 'a', 'little', 'meeting', 'of', 'the', 'speed', '.', 'May', 'I', 'have', 'a', 'ticket', 'for', 'a', 'few', 'distance', '?']
predicted trg = ['The', 'bank', 'is', 'too', 'far', 'from', 'the', 'bank', '.', 'I', 'need', 'a', 'little', '.', 'I', 'need', 'to', 'be', 'a', 'little', '.']
predicted trg = ['I', 'think', 'I', 'have', 'lost', 'lost', 'the', 'lost', ',', 'but', 'I', 'have', 'to', 'have', 'a', 'lost', 'of', 'the', 'lost', '.', 'Do', 'you', 'want', 'to', 'have', 'a', 'lost', 'of', 'the', 'lost', '?', 'I', 'need', 'to', 'be', 'out', 'with', 'the', 'lost', '.', 'Do', 'you', 'have', 'to', 'put', 'the', 'lost', 'lost', 'or', 'I']
predicted trg = ['I', "'m", 'going', 'to', 'get', 'a', 'few', 'minute', 'call', 'from', 'Busan', ',',

전체적인 Loss와 Perplexity가 가장 낮게 나온 모델은 Transformers 모델이다. 이 모델의 inference를 다시 한 번 보여주자면 아래와 같다.

In [ ]:
sen_list = [
'모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .',
'미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?',
'은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요',
'아무래도 분실 하 ㄴ 것 같 으니 분실 신고서 를 작성 하 아야 하 겠 습니다 . 사무실 로 같이 가 시 ㄹ 까요 ?',
'부산 에서 코로나 확진자 가 급증 하 아서 병상 이 부족하 아 지자  확진자 20명 을 대구 로 이송하 ㄴ다 .',
'변기 가 막히 었 습니다 .',
'그 바지 좀 보이 어 주 시 ㅂ시오 . 이거 얼마 에 사 ㄹ 수 있 는 것 이 ㅂ니까 ?',
'비 가 오 아서 백화점 으로 가지 말 고 두타 로 가 았 으면 좋 겠 습니다 .',
'속 이 안 좋 을 때 는 죽 이나 미음 으로 아침 을 대신 하 ㅂ니다',
'문 대통령 은 집단 이익 에서 벗어 나 아 라고 말 하 었 다 .',
'이것 좀 먹어 보 ㄹ 몇 일 간 의 시간 을 주 시 어요 .',
'이날 개미군단 은 외인 의 물량 을 모두 받 아 내 었 다 .',
'통합 우승 의 목표 를 달성하 ㄴ NC 다이노스 나성범 이 메이저리그 진출 이라는 또 다른 꿈 을 향하 어 나아가 ㄴ다 .',
'이번 구조 조정 이 제품 을 효과 적 으로 개발 하 고 판매 하 기 위하 ㄴ 회사 의 능력 강화 조처 이 ㅁ 을 이해 하 아 주 시 리라 생각 하 ㅂ니다 .',
'요즘 이 프로그램 녹화 하 며 많은 걸 느끼 ㄴ다 ']

In [ ]:
for sent in sen_list:
  print(sent)
  print(translate_sentence(sent, src_voc, trg_voc, model, device, max_len = 50))

모든 액체 , 젤 , 에어로졸 등 은 1 커트 짜리 여닫이 투명 봉지 하나 에 넣 어야 하 ㅂ니다 .
['All', 'liquids', ',', 'gels', 'and', 'aerosols', 'must', 'be', 'placed', 'in', 'a', 'single', 'and', 'single', 'and', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'a', 'single', 'and', 'single', 'and', 'a', 'single', 'and']
미안 하 지만 , 뒷쪽 아이 들 의 떠들 는 소리 가 커 어서 , 광화문 으로 가 아고 싶 은데 표 를 바꾸 어 주 시 겠 어요 ?
['I', "'m", 'sorry', ',', 'but', 'I', "'d", 'like', 'to', 'change', 'my', 'ticket', 'to', 'the', 'child', '.', 'Could', 'you', 'give', 'me', 'a', 'ticket', 'to', 'the', 'double', 'decker', 'bus', 'number', 'is', 'a', 'ticket', 'to', 'the', 'movie', 'ticket', ',', 'please', 'change', 'for', 'me', 'a', 'ticket', 'to', 'the', 'movie', 'beer', ',', 'and', 'reserve']
은행 이 너무 멀 어서 안 되 겠 네요 . 현찰 이 필요 하면 돈 을 훔치 시 어요
['The', 'bank', 'is', 'too', 'far', '.', 'Do', 'you', 'want', 'to', 'pay', 'too', '